# Week 9: Sampled Signal Analysis with the DFT

<font size="6"> Laboratory ? </font> <br>
<font size="3"> Last updated July 27, 2022 </font>

## <span style="color:orange;"> 00. Content </span>

### Mathematics 
- Fourier series
- Euler's formula
- Sampling and Aliasing
- Periodicity of discrete-time complex exponentials
    
### Programming Skills 
- FFT
    
### Embedded Systems 
- N/A

## <span style="color:orange;"> 0. Required Hardware </span>
- N/A

<h3 style="background-color:lightblue"> Write your name and email below: </h3>

**Name:** me 

**Email:** me @purdue.edu

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## <span style="color:orange;"> 1. Intro </span>

The signals we acquire in real-life have a finite duration. We typically assume that they "start at zero," meaning that they are equal to zero until $t=0$. After the signal duration is passed, i.e. when $t$ is greater than the duration of the signal, then we also assume that the signal is equal to zero. 

Mathematically, a finite duration signal $x(t)$ takes the form $$ x(t)=\left\{ \begin{array}{ll} x(t), & 0\leq t < \text{signal duration}, \\ 0, & \text{ else.}\end{array}  \right. $$

In order to analyze a finite duration signal, we repeat it periodically with a period $P$ that is at least as long as the duration of the signal. When $P$ is strictly greater than the duraction, we say that the signal has been "zero padded" between the repetitions. 

We assume that the repeated signal is such that it is equal to its Fourier series $$f(t)= \frac{a_0}{2}+ \sum_{k=1}^\infty a_k \cos \left(\frac{2 \pi}{P} k t \right) + b_k \sin \left(\frac{2 \pi}{P} k t  \right).$$

We are interested in analyzing the signal $f(t)$ given a digital recording of the signal. In particular, we would like to be able to find the values of the coefficients $a_k,b_k$ using the uniform sampling $ f_d[n]=f(nT)$, for some $T>0$. The parameter $T$ is called the sampling period. A uniform sampling means that the samples are equally spread out within a period. The sampling period $T$ is the time difference between two sample points. The quantity $\frac{1}{T}$ is called the sampling "frequency."   

In order to simplify the math, we rewrite $f(t)$ in terms of complex exponentials: $$ f(t)=  \sum_{k=-\infty}^\infty c_n e^{i \frac{2\pi}{P}kt} $$ where $ c_0=\frac{a_0}{2} $ and $c_n=c_{-n}^*=\frac{1}{2} (a_n- i b_n)$,  for $n>1$. 

## Exercise 1: 
Derive the above formula. (Hint: replace each sine and cosine by a sum of complex exponentials. If you do not know the sums already, find them using Euler's formula $e^{i\theta}=\cos \theta + i \sin \theta$. B'Euler up! )

We will recover the coefficients $a_k,b_k$ using the Discrete Fourier Transform, or "DFT" for short. The DFT transform a periodic signal with period $N$ into a sequence of complex numbers that is also periodic with period $N$. The numbers in the sequence are called the "DFT coefficients." 

Here is the formula for the $k^{th}$ DFT coefficient of a sampled signal $f_d[n]$ with period $N$:
$$ F[k]= \sum_{n=0}^{N-1} f_d[n] e^{-i \frac{2 \pi}{N} k n }.  $$


As we will see later, in some circumstances, namely if
- the Fourier series of $f(t)$ is finite, that is to say if $c_k=0$ for all $|k|>N_0$, and 
- $N$ is large enough,

then we have 

$$F[k]= N c_k, \text{ for all }0\leq k \leq N_0.$$ 

Therefore, in those circumstances, one can recover the Fourier series coefficients $a_k=2 (c_k+c_k^*)$, $b_k=2(c_k-c_k^*)$ of the signal $f(t)$ from the DFT coefficients of the sampled signal. The rest of the lab explores and clarifies this.


## <span style="color:orange;"> 2. Periodicity of the Sampling </span>

In order for the proposed analysis method to work, the sampled signal must be periodic. In order words, there must exist an integer $N$ such that 

$$ f_d[n+N] = f_d[n], \forall n.$$

Depending on the sampling period $T$ chosen, the sampled signal may or may not be periodic.

## Exercise 2: 
If the signal $f(t)=e^{i \frac{2\pi}{10} t }$ is sampled with a sampling period $T=2$, show that the resulting sampling $f_d[n]=f(nT)$ is periodic with period $N=5$. 

## Exercise 3: 
If the signal $f(t)=e^{i \frac{2\pi}{10} t }$ is sampled with a sampling period $T=3$, show that the resulting sampling $f_d[n]=f(nT)$ is periodic with period $N=10$. 

## Exercise 4: 
If the signal $f(t)=e^{i \frac{2\pi}{10} t }$ is sampled with a sampling period $T=\sqrt{2}$, show that the resulting sampling $f_d[n]=f(nT)$ is not periodic

In general, $f_d[n]= f(n T)$ is periodic with (integer) period $N$ if an only if $\frac{P}{T}=\frac{N}{K}$ is a rational number. For example, if $T=\frac{P}{N}$, then we acquire $N$ samples per period, and $f_d[n]$ is periodic with period $N$. Similarly, if $T=\frac{2 P}{N}$, then we acquire $N$ samples per two periods, and so $\frac{N}{2}$ samples per period. If $N$ is odd, then the period of  $f_d[n]$ is $N$ (it has to be an integer). If $N$ is even, then the period of $f_d[n]$ is $\frac{N}{2}$.

## <span style="color:orange;"> 2. Toy Example of Signal Analysis </span>

Suppose the signal recorded, once repeated periorically, is of the form $$  f(t)= a_1 \cos \left(2 \pi 100 t \right) + a_4 \cos \left(2 \pi 400 k t  \right). $$
So the signal has period $P=\frac{1}{100}$.

Given is a recording with 1000 samples per unit of time (so the sampling period is $T=\frac{1}{1000}$: $$ f_d[n] = f( n \frac{1} {1000}).$$
More specifically, the recorded signal is  

$$ \begin{aligned}
f_d[n]&= a_1 \cos \left(2 \pi 100 \frac{n} {1000} \right) + a_4 \cos \left(2 \pi 400 k \frac{n} {1000}  \right)\\
&= a_1 \cos \left(2 \pi \frac{n}{10} \right) + a_4 \cos \left(2 \pi 4 \frac{n}{10} \right). 
\end{aligned} $$

Observe that $f_d[n]$ is periodic, with period $N=10$. This is because we are taking $10$ samples per period (since $\frac{P}{T}=\frac{1000}{100}=10$).

Let us write $f_d[n]$ in terms of complex exponentials: $$ f_d[n]= \frac{a_1}{2} \left(e^{i  2 \pi \frac{n}{10}}+ e^{-i 2 \pi \frac{n}{10}}\right) + \frac{a_4}{2}\left(e^{i 2 \pi  4 \frac{n}{10}}+ e^{-i 2 \pi 4 \frac{n}{10}}\right). $$

Now let us compute the first coefficient of the DFT.  

We have 
$$ \begin{aligned}
F[1]&= \sum_{n=0}^{9} f_d[n]  e^{-i \frac{2\pi}{10} n} \\
&= \sum_{n=0}^{9} \left( \frac{a_1}{2} \left(e^{i 2 \pi \frac{n}{10}}+ e^{-i 2 \pi \frac{n}{10}}\right) 
+
\frac{a_4}{2}\left(e^{i 2 \pi  4 \frac{n}{10}}+ e^{-i 2 \pi 4 \frac{n}{10}}\right)\right)  e^{-i \frac{2\pi}{10}  n} \\
&= 10 \frac{a_1}{2}
\end{aligned}$$
where we used the fact that 
$$ \sum_{n=0}^{N-1} e^{j \frac{2\pi}{N} k_1 n } e^{j \frac{2\pi}{N} k_2 n } = \left\{ 
\begin{array}{cc}
    N &, \text{ if } k_1-k_2  \text{ is a multiple of } N  \\
   0  &, \text{ else}. 
\end{array}
\right.$$
Therefore, $a_1=2 \frac{F[1]}{10}$. So we are able to obtain the value of $a_1$ from the value of $F[1]$.

## Exercise 5: 
Give them a prerecorded signal of this form, and ask them to write code to find what is the value of $a_1$.

Similarly, we can recover $a_4$ from $F[4]$. Indeed, we have
$$ \begin{aligned}
F[4]&= \sum_{n=0}^{9} f_d[n]  e^{-i \frac{2\pi}{10} 4 n} \\
&= \sum_{n=0}^{4} \left( \frac{a_1}{2} \left(e^{i 2 \pi \frac{n}{10}}+ e^{-i 2 \pi 2 \frac{n}{10}}\right) 
+
\frac{a_4}{2}\left(e^{i 2 \pi  4 \frac{n}{10}}+ e^{-i 2 \pi 4 \frac{n}{10}}\right)\right) e^{-i \frac{2\pi}{5} 4 n} \\
&= 10 \frac{a_4}{2},
\end{aligned}$$ 
again by using the fact that 

$$ \sum_{n=0}^{N-1} e^{j \frac{2\pi}{N} k_1 n } e^{j \frac{2\pi}{N} k_2 n } = \left\{ 
\begin{array}{cc}
    N &, \text{ if } k_1-k_2  \text{ is a multiple of } N  \\
   0  &, \text{ else}. 
\end{array}
\right.$$

Therefore, $a_4=2 \frac{F[4]}{5}$. So we are able to obtain the value of $a_4$ from the value of $F[4]$.


## Exercise 6: 
Give students the same signal as before (of the given form) and ask them to obtain the value of $a_4$.

## <span style="color:orange;"> 3. Another Toy Example - with Aliasing </span>

Again, we assume thay the signal recorded is known to be of the form
$$  f(t)= a_1 \cos \left(2 \pi 100 t \right) + a_4 \cos \left(2 \pi 400 k t  \right). $$
So the signal has period $P=\frac{1}{100}$.

This time, we are given a recording with 500 samples per unit of time (so the sampling period is $T=\frac{1}{500}$): 
$$ f_d[n] = f( n \frac{1} {500}).$$
More specifically, the recorded signal is  
$$ \begin{aligned}
f_d[n]&= a_1 \cos \left(2 \pi 100 \frac{n} {500} \right) + a_4 \cos \left(2 \pi 400 k \frac{n} {500}  \right)\\
&= a_1 \cos \left(2 \pi  \frac{n}{5} \right) + a_4 \cos \left(2 \pi 4 \frac{n}{5} \right). 
\end{aligned} $$

Here $f_d[n]$ is periodic with period $N=5$. This is because we are taking $5$ samples per signal period (since $\frac{P}{T}=\frac{500}{100}=5$).

Let us write $f_d[n]$ in terms of complex exponentials:
$$ \begin{aligned}
f_d[n]&= \frac{a_1}{2} \left(e^{i 2 \pi  \frac{n}{5}}+ e^{-i 2 \pi  \frac{n}{5}}\right) 
+
\frac{a_2}{2}\left(e^{i 2 \pi  4 \frac{n}{5}}+ e^{-i 2 \pi 4 \frac{n}{5}}\right).
\end{aligned} $$

Now let us compute the first coefficient of the DFT.  

We have 
$$ \begin{aligned}
F[1]&=& \sum_{n=0}^{9} f_d[n]  e^{-i \frac{2\pi}{5} n} \\
&=& 10 \frac{a_1}{2} + 10 \frac{a_2}{2}.
\end{aligned} $$
since
$$ \sum_{n=0}^{N-1} e^{j \frac{2\pi}{N} k_1 n } e^{j \frac{2\pi}{N} k_2 n } = \left\{ 
\begin{array}{cc}
    N &, \text{ if } k_1-k_2  \text{ is a multiple of } N  \\
   0  &, \text{ else}. 
\end{array}
\right.$$

Thus, in this case, $F[1]$ does not give us the value of $a_1$ anymore, but rather the sum of $a_1$ and $a_2$. 

The reason for this is fundamental: the function $e^{i 2 \pi  4 \frac{n}{5}}$ is the same as the function $e^{-i 2 \pi   \frac{n}{5}}$. So we cannot recover the coefficients separately because the two exponential signals, once sampled, become the same signal. Indeed, the sampled signal can be written as
$$ f_d[n]= \left( \frac{a_1}{2} +  \frac{a_2}{2} \right) e^{i 2\pi  \frac{n}{5}}+ \left( \frac{a_1}{2} +  \frac{a_2}{2} \right) e^{-i 2\pi  \frac{n}{5}}.$$

Another way to see this is to observe that $\cos (2 \pi 4 \frac{n}{5})= \cos(2 \pi 4 \frac{n}{5}- 2\pi n )= \cos (-2 \pi  \frac{n}{5})=  \cos (2 \pi  \frac{n}{5})$.

So the signal $f_d[n]$ can be written as $$ \begin{aligned}
f_d[n]&= a_1 \cos \left(2 \pi  \frac{n}{5} \right) + a_4 \cos \left(2 \pi 4 \frac{n}{5} \right),\\
&= a_1 \cos \left(2 \pi  \frac{n}{5} \right)+ a_4 \cos \left(2 \pi \frac{n}{5} \right),\\
&= (a_1+a_4) \cos \left(2 \pi  \frac{n}{5} \right).
\end{aligned} $$ 
This phenomenon is called "aliasing."  

## Exercise 7: 
Check, numerically, that sampling $\cos (2\pi 100 t)$ or $\cos (2\pi 100 t)$  with a period $T=\frac{1}{500}$ yields the same sampled signal.

## <span style="color:orange;"> 4. The Theory </span>

The aliasing phenomenon is well understood. In fact, you may have heard about the Nyquist criterion: is a condition that, if satisfied, guarantees that aliasing will not uccur. In this lab, we will not dwelve further into this criterion per say but, instead, jump directly to the topic of interest, which is to analyze a signal $f(t)$ from a sampling $f_d[n]=f(nT)$. By "analysing," we mean recovering the Fourier series coefficients $c_k$.  

The theory tells us that the DFT coefficients give us the value of the Fourier series coefficients of $f(t)$ provided that we have taken "enough" samples considering the number of non-zero coefficients in the Fourier series of $f(t)$. More specifically, we have the following theorem.

## Theorem
If f(t) has a finite Fourier series with $2K+1$ terms $$f(t)=\sum_{k=-K}^K c_k e^{i \frac{2\pi}{P} t}$$ (so its maximum frequency is $K\frac{1}{P}$), and if we sample f(t) with a sampling frequency $\frac{1}{T}$ such that  $$\frac{1}{T} > 2 K\frac{1}{P}$$ then the DFT of $f_d[n]$ gives us the coefficients $a_k,b_k$ of the Fourier series of $f(t)$.  

More specifically, we have $$F[k]= N c_k= N\frac{1}{2}(a_k-ib_k), \text{ for all }  $0\leq k \leq K$$.

## Special case  
If $\frac{P}{T}=N$ is an integer (i.e., we take exactly N samples per period), and $N>2K$, then we are guaranteed that no aliasing will occur and we can recover the coefficients $c_k$ from the DFT coefficients $F[k]$ as $$F[k]= N c_k= N\frac{1}{2}(a_k-ib_k), \text{ for all } $0\leq k \leq K$$.
 

## Exercise 8: 
Show that, under the hypothesis of the theorem, we have  $F[k]= N c_k$ for all $0\leq k \leq K$.

Hint: Use the fact that $$ \sum_{n=0}^{N-1} e^{j \frac{2\pi}{N} k_1 n } e^{j \frac{2\pi}{N} k_2 n } = \left\{ 
\begin{array}{cc}
    N &, \text{ if } k_1-k_2  \text{ is a multiple of } N  \\
   0  &, \text{ else}. 
\end{array}
\right.$$

The DFT coefficients are complex numbers. Under the hypothesis of the above theorem, their magnitude $|F[k]|=N |c_k|=N \sqrt{a^2+b^2}$ tells  us how much of the frequency $k \frac{2\pi}{P}$ is in the signal, up to a constant factor $N$. So, for example, the DFT coefficient with the largest magnitude indicates the largest frequency contained in the signal.  

## Exercise 9:
- Build a signal that is a sum of sine and cosine. Vary the magnitude of the coefficients, so that one is very large compared to the other.
- Play the sound corresponding to the signal your build.
- Write a function to compute the DFT of your signal and plot its magnitude. For what value of $k_0$ does is the magnitude of $F[k]$ the largest?
- What frequency does the $k_0$ you identified correspond to? 
- Play a sound which only has the frequency $k_0$ (i.e., set all coefficients to zero, except $a_{k_0}$ and $b_{k_0}$. How does it compare to the original sound?

In the previous exercise, you had to write your own function to compute a DFT. Python has a quick command to do so: Here is the synthax.

## Exercise 10: 
Use the FFT command to compute the DFT of the sound you build in Exercise 9, and use it to find $k_0$.
  